## Model I/O

In [ ]:
from dotenv import load_dotenv
load_dotenv()

### Prompts(프롬프트)

모델에게 던지는 질문을 템플릿화 하는 기술

**주요 사용처**

1. **자동화된 입력 구성**
    - PromptTemplate을 사용하여 사용자 입력을 자동으로 구성.
    - 동일한 형식의 질문이나 대화를 대량으로 생성 가능

2. **대화형 응답**
    - ChatPromptTemplate을 통해 대화형 AI의 문맥 유지를 지원.

3. **결과 파싱**
    - Output Parsers 를 통해 LLM의 출력을 특정 포맷으로 처리하여 후속 작업 자동화.

In [ ]:
%pip install langchain-core

In [ ]:
# PromptTemplate
from langchain_core.prompts import PromptTemplate

# 템플릿 정의
template = "{product}를 홍보하기 위한 재미있고, 새로운 광고문구를 작성해 주세요."

# PromptTempalate 객체 생성
prompt = PromptTemplate(
    template=template,
    input_variables=['product']
)

# 실제 프롬프트 생성 (변수 채우기)
print(prompt.format(product='카메라'))
print(prompt.format(product='자동차'))

In [ ]:
# ChatPromptTemplate - System, Human, AI 유형별 메시지를 작성할수 있는 기능 제공
from langchain_core.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# 시스템 메세지 (역할 부여)
system_message = SystemMessagePromptTemplate.from_template("당신은 도움을 주는 친절한 챗봇입니다.")

# 휴먼 메세지 (사용자 질문)
human_message = HumanMessagePromptTemplate.from_template("질문 : {question}")

# 챗 프롬프트 합치기
messages = ChatPromptTemplate.from_messages([system_message, human_message])
print(messages)

# 포맷팅 확인
prompt = messages.format_messages(question="AI가 무엇인가요?")
print(prompt)

In [ ]:
# FewShotPromptTemplate - n-shot 프롬프팅을 위한 클래스 (말보다 '예시'로 가르치기)
from langchain_core.prompts.few_shot import FewShotPromptTemplate

# 예시 데이터
examples = [
    {"question": "2 + 2는 무엇인가요?", "answer" : "2 + 2 = 4"},
    {"question": "3 + 5는 무엇인가요?", "answer" : "3 + 5 = 8"},
]

# 예시를 어떻게 보여줄지 포맷 정의
example_prompt = PromptTemplate(
    template="Q: {question}\nA: {answer}",
    input_variables=['question', 'answer']
)

# FewShotPromptTemplate 조립
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,                   # 예시 데이터
    example_prompt=example_prompt,       # 예시 포맷
    prefix="다음 계산문제를 해결하세요",     # 지시어 (앞부분)
    suffix="Q: {question}\nA:",          # 실제 질문 (뒷부분)
    input_variables=['question']
)

prompt = few_shot_prompt.format(question="35 + 48은 무엇인가요?")
print(prompt)

### Output Parser(출력 파서)

LLM이 생성한 텍스트 출력을 특정 형식으로 변환하거나 처리하는 데 사용
- 텍스트 기반 응답을 JSON, 리스트 또는 숫자와 같은 특정 포맷으로 변환하여 후속 작업에 활용

**종류**
1. **BaseOutputParser**: Output Parsers의 기본 클래스, 커스텀 파서 구현 시 사용
2. **CommaSeparatedListOutputParser**: 콤마로 구분된 문자열을 리스트로 변환
3. **RegexParser**: 정규식을 사용해 특정 패턴을 추출하고 키-값 형태로 반환
4. **StructuredOutputParser**: 출력의 JSON 또는 구조화된 형식을 강제
5. **PydanticOutputParser**: Pydantic 모델을 기반으로 출력 검증 및 변환
6. **MarkdownOutputParser**: 마크다운 형식의 텍스트에서 데이터를 추출

In [ ]:
%pip install langchain-openai

In [ ]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_openai import ChatOpenAI

# 파서 생성
output_parser = CommaSeparatedListOutputParser()

# 지시사항 가져오기
format_instructions = output_parser.get_format_instructions()
# print(format_instructions)

# 프롬프트 템플릿에 지시사항 주입
prompt_template = PromptTemplate(
    template="{subject} 5개의 팀을 보여주세요.\n형식지정: {format_instructions}",
    input_variables=['subject'],    # 사용자 입력값 변수
    partial_variables={'format_instructions' : format_instructions} # 고정적(기본적으로 포함)으로 설정될 변수
)

# 모델 준비
model = ChatOpenAI(model_name='gpt-5-nano', temperature=0)

query = "한국의 프로야구팀"
prompt = prompt_template.format(subject=query)

# invoke로 실행
response = model.invoke(prompt)
print("원본 응답:", response.content)

# 파싱 (문자열 -> 리스트로 변환)
parsed_result = output_parser.parse(response.content)
print("파싱 결과:", parsed_result)

### Models(다양한 모델 사용하기)

In [ ]:
# ChatOpneAI

from langchain_openai import ChatOpenAI

model_gpt = ChatOpenAI(model="gpt-5-nano", temperature=1)
model_gpt.invoke("저는 아침으로 사과를 먹었습니다. 저는 아침에 무엇을 먹었을까요?")

In [ ]:
%pip install langchain-huggingface

In [ ]:
# HuggingFaceEndpoint(오픈소스 모델)
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

# 엔드포인트 설정
end_point = HuggingFaceEndpoint(
    repo_id='MLP-KTLim/llama-3-Korean-Bllossom-8B',
    task='text-generation',
    max_new_tokens=1024
)

# 챗 모델로 감싸기
hf_model = ChatHuggingFace(llm=end_point)

hf_model.invoke("저는 아침으로 사과를 먹었습니다. 저는 아침에 무엇을 먹었을까요?")

In [ ]:
%pip install -U huggingface_hub

### ModelLaboratory(모델비교)

In [ ]:
%pip install langchain-classic

In [ ]:
from langchain_classic.model_laboratory import ModelLaboratory

model_lab = ModelLaboratory.from_llms([model_gpt, hf_model])
model_lab.compare("대한민국의 겨울은 몇 월부터 몇 월인가요??")